In [ ]:
from google.colab import drive


drive.mount("/content/drive")

In [3]:
import cv2
import numpy as np


# 曇り度合い
def get_degree_of_cloudiness(img):
  img = img[:img.shape[0] // 2, :]

  hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

  lower_green = np.array([30, 64, 0])
  upper_green = np.array([90, 255, 255])

  green_mask = cv2.inRange(hsv, lower_green, upper_green)
  not_green_mask = cv2.bitwise_not(green_mask)
  image_without_green = cv2.bitwise_and(hsv, hsv, mask=not_green_mask)

  lower_blue = np.array([90, 38, 0])
  upper_blue = np.array([150, 255, 255])

  blue_mask = cv2.inRange(image_without_green, lower_blue, upper_blue)
  blue_ratio = np.count_nonzero(blue_mask) / np.count_nonzero(image_without_green)

  return blue_ratio


# 逆光度合い
def get_degree_of_backlighting(img):
  hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
  _, _, v = cv2.split(hsv)

  return v.mean()


# 視点の遠さ
def get_distance_from_camera(img):
  hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

  lower_green = np.array([30, 64, 0])
  upper_green = np.array([90, 255, 255])
  
  mask = cv2.inRange(hsv, lower_green, upper_green)

  return np.count_nonzero(mask)

In [ ]:
path_to_movie = "/content/drive/MyDrive/Colab Notebooks/Research/Raw Data/20221020/14A_1.MOV"

cap = cv2.VideoCapture(path_to_movie)

cloudiness_list = []
backlighting_list = []
distance_list = []
while True:
  ret, frame = cap.read()

  if not ret:
    break

  frame = cv2.rotate(frame, cv2.ROTATE_180)

  cloudiness_list.append(get_degree_of_cloudiness(frame))
  backlighting_list.append(get_degree_of_backlighting(frame))
  distance_list.append(get_distance_from_camera(frame))

cap.release()

print(np.median(cloudiness_list))
print(np.median(backlighting_list))
print(np.median(distance_list))